<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# [Ateliers: Technologies des grosses data](https://github.com/wikistat/Ateliers-Big-Data)

# [Reconnaissance d'Activité Humaine](https://github.com/wikistat/Ateliers-Big-Data/5-HumanActivityRecognition) ([*HAR*](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones)) en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a>  
##  Seconde partie:  apprentissage (profond) des signaux bruts  avec <a href="https://keras.io/"><img src="https://s3.amazonaws.com/keras.io/img/keras-logo-2018-large-1200.png" style="max-width: 100px; display: inline" alt="Keras"/></a>

Ce notebook présente la partie prediction de l'activité. Pour l'exploration, se référer au calepin afférent.

##  1 Introduction
###  1.1 Contexte
Les données sont issues de la communauté qui vise la reconnaissance d'activités humaines (*Human activity recognition, HAR*) à partir d’enregistrements, par exemple du gyroscope et de l'accéléromètre d'un smartphone.
Voir à ce propos l'[article](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-11.pdf) relatant un colloque de 2013.  

Les données publiques disponibles ont été acquises, décrites et analysées par [Anguita et al. (2013)](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-84.pdf). Elles sont accessibles sur le [dépôt](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) de l'University California Irvine (UCI) consacré à l'apprentissage machine ainsi que sur le site *Kaggle*.

L'archive contient les données brutes: accélérations en x, y, et z, chacun de 128 colonnes. D'autres fichiers en y soustrayant la gravité naturelle ainsi que les accélérations angulaires en x, y, et z, soit en tout 9 fichiers. Mais 6 utiles avec 6*128=768 mesures.

Les méthodes d'apprentissage sont appliquées sur ces données brutes, sans calculs préliminaires de caractéristiques (*features*).

### 1.2 Objectif
Cette deuxième étape s'intéresse aux données brutes. Est-il possible d'économiser le travail préliminaire de définition des variables métier en utilisant, par exemple, les ressources de décompositions systématiques sur une base d'ondelette ou un algorihtme d'apprentissage profond?

**Objecctif** Faire aussi bien (96% de bien classés) qu'avec les variables métier.

### 1.3 Travail à réaliser
**Attention l'accès à un environnement *GPU* est très vivement conseillé voire indispensable.**
- Modélisation, prévision de l'échantillon test par
   - Régression logistique (`Scikit-learn`)
   - Apprentissage profond en utilisant `Keras` 
       - MLP sur signaux "applatis"
       - MLP sur signaux mutlidimensionelles
       - LSTM
       - 1D Convolution
       - 2D Convolution
   
- Ajouter à ce calepin: 
    - Application des méthodes d'apprentissage classique ou non sur les coefficients des décompositions des signaux en ondelettes
    - optimisation des paramètres des différentes méthodes.
    - Améliorer l'architexture des réseaux?


## 2 Mise en place
### 2.1 Librairies et initialisation

In [1]:
import pandas as pd
import numpy as np
import os
import time
import copy
import random
import itertools

#Utils Sklearn
import sklearn.linear_model as lm
from sklearn.metrics import confusion_matrix

%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

# DEEP LEARING
import tensorflow as tf
np.random.seed(42)
tf.set_random_seed(42)

# for reproducibility
# https://github.com/fchollet/keras/issues/2280
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

import keras.models as km 
import keras.layers as kl 
import keras.layers.core as klc

Using TensorFlow backend.


### 2.2 Prise en charge des données
#### Sources

Les données sont celles originales du dépôt de l'[UCI](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones). Elle peuvent être téléchargées en cliquant [ici](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip).

Elles contiennent deux jeux de dimensions différentes, chacun partagé en apprentissage et test.

1. Multidimensionel: un individus est constitué de 9 Séries Temporelles de *dimensions* $(N, 128, 9)$
2. Unidimensionnel: Les 9 Séries Temporelles sont concaténées pour constituer un vecteur de 128x9 = 1152 variables de *dimensions* $(N, 1152)$
        
Deux objets différents sont construits pour définir la variable $Y$ réponse car les librairies `Scikit-learn` et `Keras` prennent en compte des structures différentes: 
    
1. `Scikit-Learn`  Un vecteur de dimension $(N, 1)$ avec, pour chaque individu le numéro du label de l'activité de 0 à 5.
2. `Keras` Une matrice de dimension $(N, 6)$ des indicatrices (0 ou 1) des modalités de $Y$.

#### Lecture des données

In [4]:
DATADIR_UCI = './../data_har'

SIGNALS = [ "body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z", "total_acc_x", "total_acc_y", "total_acc_z"]

def my_read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

def load_signal(data_dir, subset, signal):
    filename = f'{data_dir}/{subset}/Inertial Signals/{signal}_{subset}.txt'
    x = my_read_csv(filename).as_matrix()
    return x 

def load_signals(data_dir, subset, flatten = False):
    signals_data = []
    for signal in SIGNALS:
        signals_data.append(load_signal(data_dir, subset, signal)) 
    
    if flatten :
        X = np.hstack(signals_data)
    else:
        X = np.transpose(signals_data, (1, 2, 0))
        
    return X 

def load_y(data_dir, subset, dummies = False):
    filename = f'{data_dir}/{subset}/y_{subset}.txt'
    y = my_read_csv(filename)[0]
    
    
    if dummies:
        Y = pd.get_dummies(y).as_matrix()
    else:
        Y = y.as_matrix()
    
    return Y

Vérification des dimensions

In [5]:
#Multidimensional Data
X_train, X_test = load_signals(DATADIR_UCI, 'train'), load_signals(DATADIR_UCI, 'test')
# Flattened Data
X_train_flatten, X_test_flatten = load_signals(DATADIR_UCI, 'train', flatten=True), load_signals(DATADIR_UCI, 'test', flatten=True)

# Label Y
Y_train_label, Y_test_label = load_y(DATADIR_UCI, 'train', dummies = False), load_y(DATADIR_UCI, 'test', dummies = False)
#Dummies Y (For Keras)
Y_train_dummies, Y_test_dummies = load_y(DATADIR_UCI, 'train', dummies = True), load_y(DATADIR_UCI, 'test', dummies = True)

N_train = X_train.shape[0]
N_test = X_test.shape[0]

In [7]:
print("Dimension")
print("Données Multidimensionelles, : " + str(X_train.shape))
print("Données Unimensionelles, : " + str(X_train_flatten.shape))
print("Vecteur réponse (scikit-learn) : " + str(Y_train_label.shape))
print("Matrice réponse(Keras) : " + str(Y_train_dummies.shape))

Dimension
Données Multidimensionelles, : (7352, 128, 9)
Données Unimensionelles, : (7352, 1152)
Vecteur réponse (scikit-learn) : (7352,)
Matrice réponse(Keras) : (7352, 6)


#### Utilitaires

In [8]:
LABELS = ["WALKING","WALKING UPSTAIRS","WALKING DOWNSTAIRS","SITTING","STANDING","LAYING"]
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}


def my_confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

def _count_classes(y):
    return len(set([tuple(category) for category in y]))

## 3 Apprentissage des signaux uni-dimensionels

La base d'apprentissage est de dimension (`N_train`, 1152)

### 3.1 Régression Logistique

La Régression Logistique est une des méthodes conduisant aux meilleurs résultats sur les variables métier.

In [9]:
t_start = time.time()
model_lr = lm.LogisticRegression(verbose=1)
model_lr.fit(X_train_flatten, Y_train_label)
t_end = time.time()
t_learning = t_end-t_start
score = model_lr.score(X_test_flatten, Y_test_label)
print("\n Score With Logistic Regression on Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
lr_prediction_label = model_lr.predict(X_test_flatten)
metadata_svc = {"time_learning" : t_learning, "score" : score}
pd.DataFrame(confusion_matrix(lr_prediction_label, Y_test_label), index = LABELS, columns=LABELS)

[LibLinear]Score With Logistic Regression on Inertial Signals = 57.45, Learning time = 28.64 secondes


,WALKING,WALKING UPSTAIRS,WALKING DOWNSTAIRS,SITTING,STANDING,LAYING
WALKING,120,63,97,0,1,0
WALKING UPSTAIRS,74,218,56,23,72,27
WALKING DOWNSTAIRS,92,66,103,1,2,0
SITTING,79,32,58,397,112,0
STANDING,131,92,106,70,345,0
LAYING,0,0,0,0,0,510


**Q** Que dire de la performance?  
La performance de la régression logistique sur les données brutes n'a rien à envier aux performances obtenues sur les données métier. Il y a beaucoup de confusions, même entre les classes actives et passives qui étaient plus facile à discriminer avec les données métier. On remarque cependant que la classe `laying` est bien discriminée par rapport aux autres. 


### SVM linéaire

In [13]:
from sklearn.svm import SVC, LinearSVC
t_start = time.time()
model_svm = LinearSVC(verbose=1)
model_svm.fit(X_train_flatten, Y_train_label)
t_end = time.time()
t_learning = t_end-t_start
score = model_svm.score(X_test_flatten, Y_test_label)
print("\nScore With Linear SVC on Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
svm_prediction_label = model_svm.predict(X_test_flatten)
metadata_svc = {"time_learning" : t_learning, "score" : score}
pd.DataFrame(confusion_matrix(svm_prediction_label, Y_test_label), index = LABELS, columns=LABELS)

[LibLinear]
Score With Linear SVC on Inertial Signals = 57.41, Learning time = 59.70 secondes


/usr/local/insa/anaconda/envs/GPU/lib/python3.6/site-packages/sklearn/svm/base.py:920: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,WALKING,WALKING UPSTAIRS,WALKING DOWNSTAIRS,SITTING,STANDING,LAYING
WALKING,112,70,104,0,3,0
WALKING UPSTAIRS,80,221,60,24,83,27
WALKING DOWNSTAIRS,92,68,100,1,1,0
SITTING,101,36,69,427,123,0
STANDING,111,76,87,39,322,0
LAYING,0,0,0,0,0,510


### XGBoost

In [ ]:
from xgboost import XGBClassifier
t_start = time.time()
param=[{"n_estimators":[50,100,200]}]
model_xgb =  GridSearchCV(XGBClassifier(),param,cv=10,n_jobs=-1)
model_xgb.fit(X_train_flatten, Y_train_label)
t_end = time.time()
t_learning = t_end-t_start
score = model_xgb.score(X_test_flatten, Y_test_label)
print("\n Score With XGBoost on Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
xgb_prediction_label = model_xgb.predict(X_test_flatten)
metadata_svc = {"time_learning" : t_learning, "score" : score}
pd.DataFrame(confusion_matrix(xgb_prediction_label, Y_test_label), index = LABELS, columns=LABELS)

## A tester : 
Projeter les composantes dans les plans de l'ACP ou sur les bases d'ondelettes et runner les algo classiques dans ces bases.

### 3.2 Perceptron multicouche

Un réseau de neurones classique est appris sur les données au même format que précédemment.

**Q** Expliciter les choix des paramètres et donc la structure du réseau.  

La taille du batch n'est pas un diviseur de la dimension de l'input (7352) mais `Keras` ne le prend pas comme une contrainte. 

Le réseau est composé de deux couches perceptron simples (`Denses`) séparées par une couche `Dropout`. La couche d'entrée prend en paramètre la taille des données d'entrée : comme le réseau ne contient pas de couche de convolution, les données peuvent être passées en format 1D (un vecteur de taille 1152 des séries concaténées) comme au format 2D. Elle retourne en sortie n_hidden neurones. La fonction d'activation est la fonction `relu`, largement utilisé car convexe, ce qui facilite la rétropropagation du gradient.  

La couche `Dropout` permet à chaque époque de supprimer aléatoirement 50% des neurones en entrée. Ceci permet d'éviter le surapprentissage.  

La couche de sortie est composée de 6 neurones qui correspondent aux 6 activités. Comme chaque neurone doit avoir une sortie binaire (1 si l'activité du neurone correspond, 0 sinon), la fonction d'activation choisie est la fonction `softmax`. 

In [61]:
X_train_flatten.shape[0]

7352

In [76]:
epochs = 10
batch_size = 32
n_hidden = 32

n_features = X_train_flatten.shape[1]
n_classes=6


model_base_mlp_u =km.Sequential()
model_base_mlp_u.add(kl.Dense(n_hidden, input_shape=(n_features,),  activation = "relu"))
#model_base_mlp_u.add(kl.Dropout(0.5))
#model_base_mlp_u.add(kl.Dense(16,  activation = "relu"))
model_base_mlp_u.add(kl.Dense(n_classes, activation='softmax'))
model_base_mlp_u.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model_base_mlp_u.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 32)                36896     
_________________________________________________________________
dense_55 (Dense)             (None, 6)                 198       
Total params: 37,094
Trainable params: 37,094
Non-trainable params: 0
_________________________________________________________________


In [77]:
t_start = time.time()
model_base_mlp_u.fit(X_train_flatten,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test_flatten, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_mlp_u.evaluate(X_test_flatten, Y_test_dummies)[1] 
print("\nScore With Simple MLP on Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_mlp_u = {"time_learning" : t_learning, "score" : score}
base_mlp_u_prediction = model_base_mlp_u.predict(X_test_flatten)

my_confusion_matrix(Y_test_dummies, base_mlp_u_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 0s - loss: 0.8626 - acc: 0.6895 - val_loss: 0.7784 - val_acc: 0.7438
Epoch 2/10
7352/7352 [==============================] - 0s - loss: 0.4097 - acc: 0.8738 - val_loss: 0.5574 - val_acc: 0.8252
Epoch 3/10
7352/7352 [==============================] - 0s - loss: 0.2688 - acc: 0.9093 - val_loss: 0.4932 - val_acc: 0.8436
Epoch 4/10
7352/7352 [==============================] - 0s - loss: 0.2082 - acc: 0.9238 - val_loss: 0.4952 - val_acc: 0.8507
Epoch 5/10
7352/7352 [==============================] - 0s - loss: 0.1760 - acc: 0.9374 - val_loss: 0.3858 - val_acc: 0.8768
Epoch 6/10
7352/7352 [==============================] - 0s - loss: 0.1559 - acc: 0.9418 - val_loss: 0.4050 - val_acc: 0.8714
Epoch 7/10
7352/7352 [==============================] - 0s - loss: 0.1406 - acc: 0.9427 - val_loss: 0.3994 - val_acc: 0.8653
Epoch 8/10
7352/7352 [==============================] - 0s - loss: 0.1295 - a

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,0,0,0,27
SITTING,0,360,107,0,2,22
STANDING,0,69,458,1,2,2
WALKING,0,11,6,443,18,18
WALKING_DOWNSTAIRS,0,3,9,13,369,26
WALKING_UPSTAIRS,0,1,2,26,26,416


** Q ** : Que conclure sur ces résultats en terme de performance, de temps d'apprentissage? Comparer avec la regression logistique?  

Ce réseau de neurones relativement simple obtient de bien meilleurs résultats sur les données brutes que la régression logistique. Mise à part l'activité `Walking_upstairs`, les classes actives et passives sont assez bien discriminées. Il reste cependant des améliorations à faire intra classes (actives et passives). 

Au niveau du temps de calcul, sous l'environnement GPU l'importation des données met 25 secondes, mais l'exécution de l'algorithme est très rapide : moins de 7 secondes. Une fois les données importées, le réseau de neurones est donc plus rapide que la régression logistique. 

** Exo ** : Quelle est l'influence de l'ajout de nouvelle couche? Supression du Dropout?  

L'ajout d'une couche `Dense` avec 32 ou 16 neurones n'améliore pas la performance du réseau. Au contraire, cela contribue à augmenter le temps d'exécution, donc il ne vaut mieux ne pas la mettre. 

La suppression de la couche `Dropout` permet de diminuer la fonction perte sur l'échantillon test et on obtient de meilleurs résultats. Elle n'est donc pas nécessaire dans ce cas là, d'autant que le temps gagné par sa présence est minime (environ 1sec). 

## 4 Apprentissage des signaux multidimensionnels
Les différents signaux ne sont pas concaténées en un seul signal mais pris en compte parallèlement.

### 4.1 Perceptron multichouche
**Q** Expliciter les choix des paramètres et donc la structure du réseau.

Le réseau entrainé est le même que précédemment à la différence que les données d'entrées ne sont pas sous le même format. Les séries n'ont pas été concaténées donc on passe en entrée une matrice. `Input_dim` correspond au nombre de séries et `timesteps` à la longueur d'une série. La couche reshape permet de repasser au format 1D. 

On a supprimé la couche `Dropout` jugée inutile dans ce cas là. 



In [87]:
len(X_train[0])
len(X_train[0][0])

9

In [81]:
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

model_base_mlp =km.Sequential()
model_base_mlp.add(kl.Dense(n_hidden, input_shape=(timesteps, input_dim),  activation = "relu"))
model_base_mlp.add(kl.Reshape((timesteps*n_hidden,) , input_shape= (timesteps, n_hidden)  ))
model_base_mlp.add(kl.Dense(n_classes, activation='softmax'))

model_base_mlp.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

t_start = time.time()
model_base_mlp.fit(X_train,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_mlp.evaluate(X_test, Y_test_dummies)[1] 
print("\nScore With Simple MLP on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_mlp = {"time_learning" : t_learning, "score" : score}
base_mlp_prediction = model_base_mlp.predict(X_test)

my_confusion_matrix(Y_test_dummies, base_mlp_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 0s - loss: 0.5928 - acc: 0.7590 - val_loss: 0.6564 - val_acc: 0.7492
Epoch 2/10
7352/7352 [==============================] - 0s - loss: 0.3317 - acc: 0.8754 - val_loss: 0.5848 - val_acc: 0.7954
Epoch 3/10
7352/7352 [==============================] - 0s - loss: 0.2511 - acc: 0.9081 - val_loss: 0.5371 - val_acc: 0.8463
Epoch 4/10
7352/7352 [==============================] - 0s - loss: 0.2027 - acc: 0.9294 - val_loss: 0.5132 - val_acc: 0.8487
Epoch 5/10
7352/7352 [==============================] - 0s - loss: 0.1747 - acc: 0.9366 - val_loss: 0.5574 - val_acc: 0.8310
Epoch 6/10
7352/7352 [==============================] - 0s - loss: 0.1553 - acc: 0.9397 - val_loss: 0.5078 - val_acc: 0.8571
Epoch 7/10
7352/7352 [==============================] - 0s - loss: 0.1390 - acc: 0.9456 - val_loss: 0.5574 - val_acc: 0.8524
Epoch 8/10
7352/7352 [==============================] - 0s - loss: 0.1287 - a

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,0,0,0,27
SITTING,0,367,99,0,0,25
STANDING,0,57,433,1,0,41
WALKING,0,0,0,402,71,23
WALKING_DOWNSTAIRS,0,3,0,23,381,13
WALKING_UPSTAIRS,0,0,0,20,25,426


In [79]:
model_base_mlp.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 128, 32)           320       
_________________________________________________________________
reshape_1 (Reshape)          (None, 4096)              0         
_________________________________________________________________
dense_57 (Dense)             (None, 6)                 24582     
Total params: 24,902
Trainable params: 24,902
Non-trainable params: 0
_________________________________________________________________


### 4.2 *Long Short Time Memory (LSTM)*
Test d'un réseau avec couche LSTM avec l'idée d'appréhender la structure temporelle des données.

In [85]:
n_hidden = 32
#default stateful = False

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

batch_size=64
#else:
model_base_lstm =km.Sequential()
model_base_lstm.add(kl.LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model_base_lstm.add(kl.Dense(n_classes, activation='softmax'))

model_base_lstm.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model_base_lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dense_62 (Dense)             (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [86]:
# Default shuffle = True Meilleur avec Shuffle m True
t_start = time.time()
model_base_lstm.fit(X_train,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test, Y_test_dummies), epochs=epochs, shuffle=False)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_lstm.evaluate(X_test, Y_test_dummies)[1] 
print("Score With Simple MLP on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_lstm = {"time_learning" : t_learning, "score" : score}
base_lstm_prediction = model_base_lstm.predict(X_test)

my_confusion_matrix(Y_test_dummies, base_lstm_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 14s - loss: 1.5900 - acc: 0.2656 - val_loss: 1.3842 - val_acc: 0.4418
Epoch 2/10
7352/7352 [==============================] - 14s - loss: 1.2253 - acc: 0.4793 - val_loss: 1.2399 - val_acc: 0.5151
Epoch 3/10
7352/7352 [==============================] - 14s - loss: 1.1063 - acc: 0.5611 - val_loss: 1.1714 - val_acc: 0.5453
Epoch 4/10
7352/7352 [==============================] - 14s - loss: 1.0149 - acc: 0.6009 - val_loss: 1.0465 - val_acc: 0.5650
Epoch 5/10
7352/7352 [==============================] - 14s - loss: 0.8796 - acc: 0.6683 - val_loss: 0.9581 - val_acc: 0.6407
Epoch 6/10
7352/7352 [==============================] - 14s - loss: 0.7588 - acc: 0.7121 - val_loss: 0.8718 - val_acc: 0.6827
Epoch 7/10
7352/7352 [==============================] - 14s - loss: 0.6799 - acc: 0.7292 - val_loss: 0.8361 - val_acc: 0.6899
Epoch 8/10
7352/7352 [==============================] - 14s - loss: 0.

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,508,2,0,0,0,27
SITTING,0,316,160,3,0,12
STANDING,0,34,491,4,1,2
WALKING,0,0,6,333,114,43
WALKING_DOWNSTAIRS,0,2,27,188,189,14
WALKING_UPSTAIRS,0,0,4,126,48,293


### 3.3 Réseau avec couche convolutionelle 1D (*ConvNet*)

In [88]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

#else:
model_base_conv_1D =km.Sequential()
model_base_conv_1D.add(kl.Conv1D(32, 9, activation='relu', input_shape=(timesteps, input_dim)))
model_base_conv_1D.add(kl.MaxPooling1D(pool_size=3))
model_base_conv_1D.add(kl.Flatten())
model_base_conv_1D.add(kl.Dense(n_classes, activation='softmax'))
model_base_conv_1D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

t_start = time.time()
model_base_conv_1D.fit(X_train,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_conv_1D.evaluate(X_test, Y_test_dummies)[1] 
print("Score With Conv on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_conv = {"time_learning" : t_learning, "score" : score}
base_conv_1D_prediction = model_base_conv_1D.predict(X_test)

my_confusion_matrix(Y_test_dummies, base_conv_1D_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 13s - loss: 0.6911 - acc: 0.7516 - val_loss: 0.5756 - val_acc: 0.7883
Epoch 2/10
7352/7352 [==============================] - 0s - loss: 0.2706 - acc: 0.9138 - val_loss: 0.3730 - val_acc: 0.8802
Epoch 3/10
7352/7352 [==============================] - 0s - loss: 0.1678 - acc: 0.9410 - val_loss: 0.3152 - val_acc: 0.8901
Epoch 4/10
7352/7352 [==============================] - 0s - loss: 0.1377 - acc: 0.9479 - val_loss: 0.2729 - val_acc: 0.8921
Epoch 5/10
7352/7352 [==============================] - 0s - loss: 0.1218 - acc: 0.9523 - val_loss: 0.2535 - val_acc: 0.9087
Epoch 6/10
7352/7352 [==============================] - 0s - loss: 0.1128 - acc: 0.9529 - val_loss: 0.2329 - val_acc: 0.9135
Epoch 7/10
7352/7352 [==============================] - 0s - loss: 0.1070 - acc: 0.9562 - val_loss: 0.2485 - val_acc: 0.9165
Epoch 8/10
7352/7352 [==============================] - 0s - loss: 0.1061 - 

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,536,0,0,0,0,1
SITTING,0,404,64,0,0,23
STANDING,0,87,442,1,0,2
WALKING,0,0,0,478,15,3
WALKING_DOWNSTAIRS,0,0,0,1,417,2
WALKING_UPSTAIRS,0,0,0,12,27,432


In [89]:
model_base_conv_1D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 120, 32)           2624      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 40, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_63 (Dense)             (None, 6)                 7686      
Total params: 10,310
Trainable params: 10,310
Non-trainable params: 0
_________________________________________________________________


### 4.3 Réseau avec couche convolutionelle 2D (*ConvNet*)

In [90]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

X_train_conv = X_train.reshape(N_train, timesteps, input_dim, 1)
X_test_conv = X_test.reshape(N_test, timesteps, input_dim, 1)

#else:
model_base_conv_2D =km.Sequential()
model_base_conv_2D.add(kl.Conv2D(32, (3, 9), activation='relu', input_shape=(timesteps, input_dim, 1)))
model_base_conv_2D.add(kl.MaxPooling2D(pool_size=(2, 1)))
model_base_conv_2D.add(kl.Flatten())
model_base_conv_2D.add(kl.Dense(n_classes, activation='softmax'))
model_base_conv_2D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

t_start = time.time()
model_base_conv_2D.fit(X_train_conv,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test_conv, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_conv_2D.evaluate(X_test_conv, Y_test_dummies)[1] 
print("Score With Conv on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_conv = {"time_learning" : t_learning, "score" : score}
base_conv_2D_prediction = model_base_conv_2D.predict(X_test_conv)

my_confusion_matrix(Y_test_dummies, base_conv_2D_prediction)

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 0s - loss: 0.7712 - acc: 0.7144 - val_loss: 0.6639 - val_acc: 0.7231
Epoch 2/10
7352/7352 [==============================] - 0s - loss: 0.3920 - acc: 0.8589 - val_loss: 0.5642 - val_acc: 0.7808
Epoch 3/10
7352/7352 [==============================] - 0s - loss: 0.2775 - acc: 0.9027 - val_loss: 0.4799 - val_acc: 0.8215
Epoch 4/10
7352/7352 [==============================] - 0s - loss: 0.2190 - acc: 0.9259 - val_loss: 0.4532 - val_acc: 0.8232
Epoch 5/10
7352/7352 [==============================] - 0s - loss: 0.1826 - acc: 0.9350 - val_loss: 0.4056 - val_acc: 0.8544
Epoch 6/10
7352/7352 [==============================] - 0s - loss: 0.1609 - acc: 0.9414 - val_loss: 0.4002 - val_acc: 0.8670
Epoch 7/10
7352/7352 [==============================] - 0s - loss: 0.1461 - acc: 0.9470 - val_loss: 0.3827 - val_acc: 0.8565
Epoch 8/10
7352/7352 [==============================] - 0s - loss: 0.1356 - a

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,0,0,0,27
SITTING,0,323,160,0,0,8
STANDING,0,35,495,0,0,2
WALKING,0,0,0,462,19,15
WALKING_DOWNSTAIRS,0,0,0,60,343,17
WALKING_UPSTAIRS,0,0,0,28,28,415


In [91]:
model_base_conv_2D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 1, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 1, 32)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2016)              0         
_________________________________________________________________
dense_64 (Dense)             (None, 6)                 12102     
Total params: 12,998
Trainable params: 12,998
Non-trainable params: 0
_________________________________________________________________


**Objectif** trouver la meilleure architecture.

**Attention au sur-apprentissage** A force de rechercher la meilleure architecture en minimisant l'erreur sur l'échantillon test, celle finalement trouvée peut y être très adaptée réduisant ainsi la capacité de généralisation. Il serait prudent de multiplier le découpage de l'échantillon par validation croisée *Monte Carlo*.